In [1]:
from pyspark.sql import SparkSession,DataFrame
from functools import reduce
import json
import timeit
import pyspark.sql.functions as f 

spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.225:7077") \
        .appName("Weak_scaling_3")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.cores.max",3)\
        .getOrCreate()

In [2]:
%%time
start_time_0 = timeit.default_timer()

df = spark_session.read.json("hdfs://192.168.2.225:9000/reddit/RC_2012*")

elapsed_0 = timeit.default_timer() - start_time_0

print("Time elapsed: ", elapsed_0, "s")

Time elapsed:  399.28149824996945 s
CPU times: user 108 ms, sys: 19.4 ms, total: 128 ms
Wall time: 6min 39s


In [3]:
df.count()

50247843

In [4]:
initial_start_time = timeit.default_timer()

if 'author_cakeday' in df.columns:
        current_df = df.drop('author_cakeday')
        
df = df.drop('author_flair_css_class','author_flair_text','can_gild','distinguished','edited','id','is_submitter','link_id','parent_id','permalink','retrieved_on','stickied','subreddit_id')

df = df.filter((df.body != '[deleted]'))


elapsed = timeit.default_timer() - initial_start_time



print("Time elapsed: ", elapsed, "s")

Time elapsed:  0.4089415839407593 s


In [5]:
%%time
start_time_1 = timeit.default_timer()

#df.select("author").show()
df.groupBy('subreddit').count().sort("count", ascending = False).show(10)

elapsed_1 = timeit.default_timer() - start_time_1

print("Time elapsed: ", elapsed_1,"s")

+-------------------+-------+
|          subreddit|  count|
+-------------------+-------+
|          AskReddit|6040651|
|              funny|2462003|
|               pics|1938503|
|             gaming|1544051|
|            atheism|1441659|
|           politics|1239044|
|                WTF|1221038|
|fffffffuuuuuuuuuuuu| 954427|
|              trees| 939739|
|               IAmA| 845624|
+-------------------+-------+
only showing top 10 rows

Time elapsed:  267.0302400160581 s
CPU times: user 60 ms, sys: 26.7 ms, total: 86.7 ms
Wall time: 4min 27s


In [6]:
%%time
start_time_2 = timeit.default_timer()

df = df.withColumn('wordCount', f.size(f.split(f.col('body'), ' ')))
#df.show()

df.groupBy('wordCount').count().sort("count", ascending = False).show(10)   

elapsed_2 = timeit.default_timer() - start_time_2


print("Time elapsed: ", elapsed_2,"s")

+---------+-------+
|wordCount|  count|
+---------+-------+
|        5|1822591|
|        6|1815501|
|        4|1757389|
|        7|1746053|
|        8|1660364|
|        1|1613596|
|        3|1605922|
|        9|1563206|
|       10|1466735|
|        2|1437728|
+---------+-------+
only showing top 10 rows

Time elapsed:  367.8059299539309 s
CPU times: user 93.6 ms, sys: 25.7 ms, total: 119 ms
Wall time: 6min 7s


In [7]:
%%time
start_time_3 = timeit.default_timer()

df_stats = df.select(f.mean(f.col('wordCount')).alias('mean')).collect()  #

mean = df_stats[0]['mean']

print("Average wordcount in comment: ", mean, " words.")


elapsed_3 = timeit.default_timer() - start_time_3
print("Time elapsed: ", elapsed_3,"s")

Average wordcount in comment:  32.311939209162745  words.
Time elapsed:  362.2791720420355 s
CPU times: user 84.9 ms, sys: 47 ms, total: 132 ms
Wall time: 6min 2s


In [8]:
total_elapsed = timeit.default_timer() - initial_start_time

print("Total time elapsed: ", total_elapsed, "s")

Total time elapsed:  997.6612992479932 s


In [9]:
spark_session.stop()